# Deep Learning with Python
5.1 Introduction to convnets

Antes de mais nada os notebooks aqui mostrado tiveram como base/foram retirados dos seguintes repositórios: 
 > https://github.com/fchollet/deep-learning-with-python-notebooks 
 
 
 > https://github.com/cdfmlr/Deep-Learning-with-Python-Notebooks
 
 Sugiro fortemente que consultem os códigos originais e em caso de dúvida podem me contatar para conversarmos. 

### Introdução a redes neurais de courger

Redes neurais de bobinas lidam muito bem com problemas de visão computacional.

Vamos começar com um exemplo do mais simples processamento de rede neural courger MNIST completa capítulo ii da rede de conexão completa:

In [1]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(10, activation="softmax"))

Aqui usamos a camada Conv2D para input_shape (image_height, image_width, image_channels) neste formato.

A saída das camadas Conv2D e Max Pooling 2D é do tipo: (altura, largura, canais), na qual altura e largura são reduzidas camada por camada, e os canais são controlados pelo primeiro parâmetro de Conv2D.

Nas últimas três camadas, transformamos as dezenas da última camada Conv2D (3, 3, 64) no vetor de características através da camada de Flatenning.  E logo depois as camadas densas com as funções de ativação que já vimos aqui. 

Por fim, veja a estrutura do modelo:

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

Bem, a rede é construída assim, e então é treinada.

In [3]:
#Treine a rede neural do rolo na imagem MNIST
import datetime
import tensorflow as tf

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


model.compile(optimizer='rmsprop',
              loss="categorical_crossentropy",
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 34s 564us/sample - loss: 0.1743 - accuracy: 0.9453
Epoch 2/5
60000/60000 [==============================] - 36s 605us/sample - loss: 0.0484 - accuracy: 0.9849
Epoch 3/5
60000/60000 [==============================] - 38s 636us/sample - loss: 0.0323 - accuracy: 0.9904
Epoch 4/5
60000/60000 [==============================] - 40s 672us/sample - loss: 0.0251 - accuracy: 0.9924
Epoch 5/5
60000/60000 [==============================] - 40s 670us/sample - loss: 0.0206 - accuracy: 0.9937


Vamos dar uma olhada nos resultados no conjunto de testes:

In [4]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(test_loss, test_acc)

10000/10000 - 2s - loss: 0.0276 - accuracy: 0.9916
0.02759703517325979 0.9916


Redes neurais de carretel

A densa camada de conexão que usamos anteriormente aprendeu o padrão global em todo o espaço de recurso de entrada (todos os pixels no MNIST); Ou seja, Dense aprende toda a imagem, e Conv aprende a parte da imagem, como a janela 3x3 no código que acabamos de escrever.

Esta rede neural cambaleante tem duas propriedades:

O padrão aprendido pelas redes neurais converse é translacional: depois que a rede neural convutiva aprende um padrão e vê o mesmo padrão em outros lugares, reconhece que aprendeu isso e não precisa aprender novamente. E para a rede de Dense, mesmo que você encontre a mesma parte local, você ainda tem que reaprender. Esta propriedade permite que a rede neural do rolo use dados de forma eficiente, e requer menos amostras de treinamento para aprender uma melhor representação de dados de generalização (uma a uma, não por mapeamento como um todo).

A rede neural do rolo pode aprender a hierarquia espacial dos padrões: depois que a rede neural do rolo aprendeu um pequeno padrão local na primeira camada, a próxima camada pode soletrar padrões maiores com esses pequenos locais. Então, mais algumas camadas, a rede neural do rolo pode aprender mais e mais complexos, mais e mais conceitos visuais abstratos.

A camada de carretel


As dezenas 3D de posições que usamos para representar a imagem no exemplo que acabamos de usar incluem dois eixos espaciais de altura, largura e uma profundidade de eixo de profundidade (também conhecido como eixo de canais), para imagens RGB, a dimensão do eixo de profundidade é 3, representando 3 cores, e para MNIST esta imagem em escala de cinza, a profundidade é 1, com apenas um número para representar o valor em escala de cinza. O resultado deste volume 3D e da operação de revolta feita nele é chamado de mapa de recursos.

A operação do rolo extrai um pequeno pedaço do gráfico de recurso de entrada e impõe a mesma transformação em todos eles para obter o gráfico de recursos de saída. O gráfico do recurso de saída ainda é um campo 3D: com largura e altura, sua profundidade pode ser arbitrária, a profundidade é um argumento para a camada, e cada canal no eixo de profundidade representa um filtro (filtro). O filtro codifica um aspecto dos dados de entrada, por exemplo, um filtro pode codificar o conceito de "um rosto está incluído na entrada".

No exemplo do MNIST há pouco, a primeira camada de carretel aceita um gráfico de recurso de entrada com dimensões (28, 28, 1) e produz um gráfico de características com dimensões (26, 26, 32). Esta saída contém 32 filtros, e o canal em cada eixo de profundidade contém um valor de 26x26, chamado mapa de resposta do filtro à entrada, que representa o resultado da operação do filtro em diferentes locais da entrada. É por isso que os gráficos de recursos são chamados de gráficos de recursos: cada dimensão do eixo de profundidade é um recurso (ou filtro), e a saída 2D outshow é um gráfico espacial bidimensional da resposta do filtro na entrada.